In [2]:
import pandas as pd
import numpy as np
import os
import geopandas
import geopy

# function get_feat_loc

## get the query for the inputed city

## geolocation by name, get IDnumber

NOTE: Search by area using relation or way id. First lookup OSM id in osm.org. The area id needs to be OSM id + 2400000000 for ways and OSM id + 3600000000 for relations. See https://wiki.osm.org/wiki/Overpass_API/Overpass_QL#By_area_.28area.29

In [4]:
from livablestreets.osm_query import query_params_osm

In [15]:
from geopy.geocoders import Nominatim
import overpy

city_name = "berlin"

# Geocoding request via Nominatim
geolocator = Nominatim(user_agent="city_compare")
geo_results = geolocator.geocode(city_name, exactly_one=False, limit=3)

# Searching for relation in result set
for r in geo_results:
    print(r.address, r.raw.get("osm_type"))
    if r.raw.get("osm_type") == "relation":
        city = r
        break


# Calculating area id
area_relid = int(city.raw.get("osm_id")) + 3600000000
area_wayid = int(city.raw.get("osm_id")) + 2400000000
area_osmid = int(city.raw.get("osm_id")) 
print(area_osmid)

Berlin, Deutschland relation
62422


## function param string

###  Dict for params

In [16]:
public_transport = {'amenity':['bus_station'],
                    'bus_bay':'',
                   'highway':['bus_stop','platform'],
                   'public_transport':['stop_position','platform','station','stop_area'],
                   'railway':['station','tram_stop', 'subway_entrance']}

bike_infraestructure =  {'amenity':['bicycle_parking', 'bicycle_repair_station', 'bicycle_rental']}

#Queries for Cyclelanes and pedestrians
cycle_paths = {'bicycle':['designated'],
              'highway':['cycleway']}

#Queries for social life
eating = {'amenity':['cafe','restaurant', 'food_court', 'ice_cream']}

night_life = { 'amenity':['bar','pub','biergarten', 'nightclub', 'swingerclub', 'casino']}

culture = {'amenity':['social_centre','theatre','public_bookcase',
                      'fountain', 'events_venue', 'community_centre', 'cinema',
                      'arts_centre', 'conference_centre']}

community = {'office': ['association','charity', 'coworking',
                        'educational_institution', 'employment_agency', 'foundation',
                        'ngo', 'political_party', 'research']}

#Queries for acitvities
health_care = {'amenity':['baby_hatch','clinic','dentist',
                      'doctors', 'hospital', 'nursing_home', 'pharmacy',
                      'social_facility', 'veterinary']}

public_service = {'amenity':['courthouse','fire_station','police',
                      'post_box', 'post_office', 'townhall']}

education = {'amenity':['college','kindergarten','language_school',
                      'library', 'music_school', 'school', 'university']}

economic = {'amenity':['atm', 'bank', 'bureau_de_change']}

#Queries for comfort
comfort_sports = {'amenity':['bbq','bench','dog_toilet',
                      '	drinking_water', 'give_box', 'shelter', 'shower',
                      'toilets', 'water_point', 'watering_place']}

leisure_sports = {'leisure':['bandstand','bird_hide','dog_park',
                      'firepit', 'swimming_pool', 'stadium', 'sports_centre',
                      'pitch', 'picnic_table', 'fitness_centre'],
                 'historic':''}

#querie for areas
leirsure_areas = {'leisure':['park','garden','swimming_area',
                      'playground', 'nature_reserve', 'marina'],
                 'landuse':['forest']}


In [41]:
#list_keys = public_transport, bike_infraestructure, cycle_paths, eating, night_life, culture, community, health_care, public_service, education, economic, comfort_sports, leisure_sports,leirsure_areas]

In [42]:
keys_values_osm = {'amenity':['bus_station'],
                    'bus_bay':'',
                   'highway':['bus_stop','platform'],
                   'public_transport':['stop_position','platform','station','stop_area'],
                   'railway':['station','tram_stop', 'subway_entrance']}

In [43]:
def param_nwr(keys):
    '''converts the dict into a string, returns a str'''
    osm_keys = ''
    for k,val in keys.items():
        for v in val:
            osm_keys += f"""nwr['{k}'='{v}'](area.bound);"""
    return osm_keys

In [44]:
param_nwr(keys_values_osm)

"nwr['amenity'='bus_station'](area.bound);nwr['highway'='bus_stop'](area.bound);nwr['highway'='platform'](area.bound);nwr['public_transport'='stop_position'](area.bound);nwr['public_transport'='platform'](area.bound);nwr['public_transport'='station'](area.bound);nwr['public_transport'='stop_area'](area.bound);nwr['railway'='station'](area.bound);nwr['railway'='tram_stop'](area.bound);nwr['railway'='subway_entrance'](area.bound);"

## min max lon lat for city Requests

In [45]:
import requests

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = f"""
                [out:json][timeout:25];
                rel({area_osmid});
                out geom;
                """
response = requests.get(overpass_url, 
                            params={'data': overpass_query})
res = response.json()

In [67]:
res['elements'][0]

{'type': 'relation',
 'id': 62422,
 'bounds': {'minlat': 52.3382448,
  'minlon': 13.088345,
  'maxlat': 52.6755087,
  'maxlon': 13.7611609},
 'members': [{'type': 'node',
   'ref': 240109189,
   'role': 'admin_centre',
   'lat': 52.5170365,
   'lon': 13.3888599},
  {'type': 'way',
   'ref': 50291800,
   'role': 'outer',
   'geometry': [{'lat': 52.5677149, 'lon': 13.1502794},
    {'lat': 52.5672981, 'lon': 13.1501373},
    {'lat': 52.5670656, 'lon': 13.1500566},
    {'lat': 52.5667597, 'lon': 13.1498626},
    {'lat': 52.5663408, 'lon': 13.1494219},
    {'lat': 52.565903, 'lon': 13.1489672},
    {'lat': 52.5656087, 'lon': 13.1486468},
    {'lat': 52.5655468, 'lon': 13.1485713},
    {'lat': 52.56519, 'lon': 13.1481356},
    {'lat': 52.565007, 'lon': 13.1480074},
    {'lat': 52.5647027, 'lon': 13.1476998},
    {'lat': 52.5644032, 'lon': 13.1475134},
    {'lat': 52.5639754, 'lon': 13.1474452},
    {'lat': 52.5638866, 'lon': 13.1474478},
    {'lat': 52.5634029, 'lon': 13.1474654},
    {'lat'

## overpass call for features Overpy

In [7]:
# Excecuting overpass call
api = overpy.Overpass()
result = api.query(f"""
    area(%s)->.bound;
    (
      {param_nwr(keys_values_osm)}
    );
    out center;
    """ % area_relid)

# Printing no. of pubs in nodes and ways
print("Amenities in nodes: %d" % len(result.nodes))
print("Amenities in ways: %d" % len(result.ways))
print("Amenities in areas: %d" % len(result.areas))

Amenities in nodes: 64
Amenities in ways: 8
Amenities in areas: 0


In [8]:
coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in result.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in result.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in result.relations]
coords

[(-4.7510922, 37.8974161),
 (-4.7576937, 37.8931537),
 (-4.8059032, 37.8863842),
 (-4.802714, 37.8863101),
 (-4.7220061, 37.9134638),
 (-4.7170172, 37.9150974),
 (-4.7223495, 37.9146319),
 (-4.7521339, 37.8820416),
 (-4.7816133, 37.8790411),
 (-4.7837238, 37.8808516),
 (-4.7789132, 37.8783106),
 (-4.7829999, 37.8827895),
 (-4.7815339, 37.8806778),
 (-4.7749311, 37.8838385),
 (-4.7822442, 37.8840124),
 (-4.788114, 37.8793787),
 (-4.7819915, 37.8841009),
 (-4.7710638, 37.8845382),
 (-4.779732, 37.8801702),
 (-4.7800873, 37.8842692),
 (-4.7922515, 37.9095783),
 (-4.7792381, 37.872689),
 (-4.7799265, 37.8822715),
 (-4.7741714, 37.8807151),
 (-4.7761913, 37.8858709),
 (-4.7759259, 37.8854521),
 (-4.7827376, 37.8777361),
 (-4.7802741, 37.8841268),
 (-4.7814581, 37.8836883),
 (-4.7897324, 37.9027705),
 (-4.7774696, 37.8792268),
 (-4.7665657, 37.877129),
 (-4.7886834, 37.8787908),
 (-4.7806372, 37.8801852),
 (-4.7757151, 37.8753902),
 (-4.7760108, 37.8799036),
 (-4.7768312, 37.8796157),
 (-4.7

In [4]:
import utm
import pandas as pd
import numpy as np

In [3]:
import requests

url_geoportal_UTM = 'https://fbinter.stadt-berlin.de/fb/wfs/data/senstadt/s_wfs_baumbestand?service=WFS&request=GetFeature&version=2.0.0&typeNames=fis%3As_wfs_baumbestand&outputFormat=application%2Fjson&resultType=results&count=10'

url_geoportal_UTM_full = 'https://fbinter.stadt-berlin.de/fb/wfs/data/senstadt/s_wfs_baumbestand?service=WFS&request=GetFeature&version=2.0.0&typeNames=fis%3As_wfs_baumbestand&outputFormat=application%2Fgeo%2Bjson&resultType=results'

response = requests.get(url_geoportal_UTM_full).json()

In [15]:
res = response['features']
res

[{'type': 'Feature',
  'id': '00008100:000bbafb',
  'geometry': {'type': 'Point',
   'coordinates': [394532.2558000004, 5811460.9911]},
  'properties': {'baumid': '00008100:000bbafb',
   'standortnr': '93',
   'kennzeich': '01414',
   'namenr': 'Fritz-Reuter-Allee',
   'art_dtsch': 'Pyramiden-Hainbuche',
   'art_bot': "Carpinus betulus 'Fastigiata'",
   'gattung_deutsch': 'HAINBUCHE',
   'gattung': 'CARPINUS',
   'strname': 'Fritz-Reuter-Allee',
   'hausnr': 'gü.111/113',
   'zusatz': '',
   'pflanzjahr': '1975',
   'standalter': '47.0',
   'kronedurch': '',
   'stammumfg': '109',
   'baumhoehe': '15.0',
   'bezirk': 'Neukölln',
   'eigentuemer': 'Land Berlin'}},
 {'type': 'Feature',
  'id': '00008100:000bbafd',
  'geometry': {'type': 'Point',
   'coordinates': [394541.9243000001, 5811473.2172]},
  'properties': {'baumid': '00008100:000bbafd',
   'standortnr': '91',
   'kennzeich': '01414',
   'namenr': 'Fritz-Reuter-Allee',
   'art_dtsch': 'Berg-Ahorn, Weiss-Ahorn',
   'art_bot': 'Ace

In [26]:
response_features = response['features']
i=0
features_tmp = []
lat_lon = []

for feature in response_features:
    np.array(features_tmp.append(feature['geometry']['coordinates']))
    lat_lon.append(utm.to_latlon(feature['geometry']['coordinates'],32,32, 'U'))
#df1 = pd.DataFrame(features_tmp, columns=['lat','lon'])

OutOfRangeError: easting out of range (must be between 100,000 m and 999,999 m)

In [8]:
for i in df1['X utm','Y utm']:
    print(i)

KeyError: ('X utm', 'Y utm')